In [27]:
import pandas as pd
import pickle
import numpy as np
from scipy.sparse import coo_matrix
import os
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [3]:
users = pd.read_csv('../data/users.csv')
games =  pd.read_parquet('../data/cleaned/games_cleaned.parquet')
test_df = pd.read_parquet('../data/cleaned/test_recommendations.parquet')
train_df = pd.read_parquet('../data/cleaned/train_recommendations.parquet')

In [4]:
unique_users = train_df['user_id'].unique()
unique_items = games['app_id'].unique()

user_to_idx = {uid: idx for idx, uid in enumerate(unique_users)}
item_to_idx = {iid: idx for idx, iid in enumerate(unique_items)}
idx_to_user = {idx: uid for uid, idx in user_to_idx.items()}
idx_to_item = {idx: iid for iid, idx in item_to_idx.items()}


In [5]:
model = pickle.load(open('../models/als_model.pkl', 'rb'))
interaction_matrix = pickle.load(open('../models/interaction_matrix.pkl', 'rb'))
tfidf_matrix = pickle.load(open('../models/tfidf_matrix.pkl', 'rb'))

In [35]:
def get_collaborative_recommendations_for_user(user_id, N=10, filter_already_recommended=True):
    # Verifica se l'user_id esiste nel dizionario
    if user_id not in user_to_idx:
        return []
    
    # Ottieni l'indice dell'utente
    user_idx = user_to_idx[user_id]
    
    # Genera le raccomandazioni (item_idx, score)
    recommended_items = model.recommend(user_idx, interaction_matrix[user_idx], N=N)
    
    if not filter_already_recommended:
        # Se non vogliamo filtrare gli articoli già visti, restituiamo direttamente
        return [(idx_to_item[idx], score) for idx, score in recommended_items]
    
    # Filtra gli articoli già visti dall'utente
    recommended_items_filtered = []
    for idx, score in zip(recommended_items[0], recommended_items[1]):
        # Controlla se l'articolo è già stato visto dall'utente (se è già nella matrice di interazione)
        if interaction_matrix[user_idx, idx] == 0:  # 0 significa che l'utente non ha interagito
            recommended_items_filtered.append((idx_to_item[idx], score))
    
    # Ordina per punteggio (in ordine decrescente)
    recommended_items_filtered = sorted(recommended_items_filtered, key=lambda x: x[1], reverse=True)
    
    return recommended_items_filtered

# Test the recommendation function
user_id = 12584260
recommended_items = get_collaborative_recommendations_for_user(user_id, N=10)
for item, score in recommended_items:
    print(f"Item ID: {item}, Score: {score:.4f}")

Item ID: 431960, Score: 0.7049
Item ID: 105600, Score: 0.6640
Item ID: 262060, Score: 0.5765
Item ID: 570, Score: 0.5652
Item ID: 242760, Score: 0.5574
Item ID: 1172470, Score: 0.5185
Item ID: 367520, Score: 0.5102
Item ID: 374320, Score: 0.5035
Item ID: 814380, Score: 0.4990
Item ID: 49520, Score: 0.4566


In [10]:
def recommend_content_based(game_id, N=10):
    
    game_idx = item_to_idx[game_id]
    
    # Estrai il vettore del gioco per cui vogliamo le raccomandazioni
    game_vector = tfidf_matrix[game_idx]
    
    # Calcola la similarità coseno tra il vettore del gioco richiesto e tutti gli altri
    similarity_scores = cosine_similarity(game_vector.reshape(1, -1), tfidf_matrix)[0]
    
    # Ordina i giochi per similarità decrescente
    similar_indices = similarity_scores.argsort()[::-1][1:N+1]  # Ignora il gioco stesso
    
    # Ottieni gli app_id dei giochi più simili e le loro distanze (similarità)
    recommended_game_ids = games['app_id'].iloc[similar_indices].tolist()
    recommended_distances = similarity_scores[similar_indices].tolist()
    
    return recommended_game_ids, recommended_distances


# Ora fuori dalla funzione, otteniamo i nomi dei giochi:
current_game_name = games[games['app_id'] == 1078730]['title'].values[0]
recommended_games, recommended_distances = recommend_content_based(1078730, N=10)
# Ottieni i nomi dei giochi raccomandati
recommended_game_names = games[games['app_id'].isin(recommended_games)]['title'].tolist()

# Stampa il risultato
print(f"Game being considered: {current_game_name}")
print("Recommended games:")
for game in recommended_game_names:
    print(f"- {game}")

Game being considered: Soccer Manager 2020
Recommended games:
- Pro 11 - Football Manager Game
- Soccer Manager 2021
- WE ARE FOOTBALL
- New Star Manager
- Soccer Manager 2022
- Football Manager 2023
- Futuball - Future Football Manager Game
- Club Manager 2016
- Football Manager 2020 In-game Editor
- Crazy Soccer: Football Stars


In [72]:
content_train_dataset = train_df.copy()
content_inference_dataset = pd.concat([train_df, test_df])

In [73]:
def get_content_recommendations_for_user(user_id, N=10, mode='train'):
    recommended_games_dict = {}
    # Itera su tutti i giochi con cui l'utente ha interagito
    if mode == 'train':
        dataset = content_train_dataset
    elif mode == 'inference':
        dataset = content_inference_dataset
    else:
        raise ValueError("Invalid mode. Use 'train' or 'inference'.")
    for game_id in dataset[dataset['user_id'] == user_id]['app_id']:
        # Ottieni le raccomandazioni per il gioco corrente
        recommended_games, recommended_similarity = recommend_content_based(game_id, N=10)
        
        # Aggiungi le raccomandazioni al dizionario, escludendo il gioco stesso
        for rec_game_id, similarity in zip(recommended_games, recommended_similarity):
            if rec_game_id != game_id:  # Escludi il gioco stesso
                if rec_game_id not in recommended_games_dict:
                    recommended_games_dict[rec_game_id] = similarity
                else:
                    # Mantieni solo la similarità più alta per ogni gioco
                    recommended_games_dict[rec_game_id] = max(recommended_games_dict[rec_game_id], similarity)

    # Ordina i giochi in base alla similarità e prendi i primi N
    sorted_recommendations = sorted(recommended_games_dict.items(), key=lambda x: x[1], reverse=True)

    # Torna i primi N giochi distinti
    top_n_games = sorted_recommendations[:N]
    
    return top_n_games


# Test the recommendation function
user_id = 2212396
recommended_items = get_content_recommendations_for_user(user_id, N=10, mode='train')
for item, score in recommended_items:
    print(f"Item ID: {item}, Score: {score:.4f}")

In [70]:
def get_weighted_recommendations(collaborative_recommendations, content_recommendations, N=10):

    collaborative_weight = 0.6
    content_weight = 0.4

    # Re-weigh the scores
    weighted_recommendations = {}
    for collaborative_item, collaborative_score in collaborative_recommendations:
        weighted_recommendations[collaborative_item] = collaborative_score * collaborative_weight
    for content_item, content_score in content_recommendations:
        if content_item in weighted_recommendations:
            weighted_recommendations[content_item] += content_score * content_weight
        else:
            weighted_recommendations[content_item] = content_score * content_weight

    # Sort by score
    sorted_weighted_recommendations = sorted(weighted_recommendations.items(), key=lambda x: x[1], reverse=True)
    return sorted_weighted_recommendations[:N]



In [63]:
def get_recommendations(user_id, mode='train', N=10):

    collaborative_recommendations = get_collaborative_recommendations_for_user(user_id, N=N)
    content_recommendations = get_content_recommendations_for_user(user_id, N=N, mode=mode)

    return collaborative_recommendations, content_recommendations


# Test the recommendation function
user_id = 12584260
collaborative_recommendations, content_recommendations = get_recommendations(user_id)
weighted_recommendations = get_weighted_recommendations(collaborative_recommendations,content_recommendations, N=10)
print("Collaborative Recommendations:")
for item, score in collaborative_recommendations:
    print(f"Item ID: {item}, Score: {score:.4f}")
print("\nContent Recommendations:")
for item, score in content_recommendations:
    print(f"Item ID: {item}, Score: {score:.4f}")

print("\nWeighted Recommendations:")
for item, score in weighted_recommendations:
    print(f"Item ID: {item}, Score: {score:.4f}")




Collaborative Recommendations:
Item ID: 431960, Score: 0.7049
Item ID: 105600, Score: 0.6640
Item ID: 262060, Score: 0.5765
Item ID: 570, Score: 0.5652
Item ID: 242760, Score: 0.5574
Item ID: 1172470, Score: 0.5185
Item ID: 367520, Score: 0.5102
Item ID: 374320, Score: 0.5035
Item ID: 814380, Score: 0.4990
Item ID: 49520, Score: 0.4566

Content Recommendations:
Item ID: 219740, Score: 1.0000
Item ID: 1238810, Score: 1.0000
Item ID: 1233220, Score: 0.9094
Item ID: 1752910, Score: 0.8515
Item ID: 457840, Score: 0.8415
Item ID: 1237980, Score: 0.8392
Item ID: 1238762, Score: 0.8249
Item ID: 665380, Score: 0.8113
Item ID: 457841, Score: 0.7943
Item ID: 417461, Score: 0.7824

Weighted Recommendations:
Item ID: 431960, Score: 0.4230
Item ID: 219740, Score: 0.4000
Item ID: 1238810, Score: 0.4000
Item ID: 105600, Score: 0.3984
Item ID: 1233220, Score: 0.3638
Item ID: 262060, Score: 0.3459
Item ID: 1752910, Score: 0.3406
Item ID: 570, Score: 0.3391
Item ID: 457840, Score: 0.3366
Item ID: 123798

In [64]:
# Ottieni gli utenti presenti nel test set e nel train set come set per una ricerca più veloce
test_users_set = set(test_df['user_id'].unique())
train_users_set = set(train_df['user_id'].unique())

In [65]:
known_users = list(test_users_set & train_users_set)  # Intersezione come lista
cold_start_users = list(test_users_set - train_users_set)

In [66]:
# Conta il numero di righe per ogni user_id nel test set
user_counts = test_df.groupby('user_id').size()

# Filtra gli utenti che hanno almeno 10 righe
known_users_filtered = [user_id for user_id in known_users if user_counts[user_id] >= 10]

# Stampa gli utenti filtrati
print(f"Utenti noti con almeno 10 righe nel test set: {len(known_users_filtered)}")


Utenti noti con almeno 10 righe nel test set: 15029


In [67]:
import numpy as np

# Funzione per calcolare la precisione
def calculate_precision(user_id, recommended_items, test_df):
    # Ottieni gli articoli effettivi con cui l'utente ha interagito nel test set (dove is_recommended == 1)
    actual_items = test_df[(test_df['user_id'] == user_id) & (test_df['is_recommended'] == 1)]['app_id']
    
    # Confronta gli articoli raccomandati con quelli effettivi
    recommended_item_ids = [item[0] for item in recommended_items]
    
    # Calcola la precisione come frazione di articoli raccomandati che sono effettivamente rilevanti
    relevant_recommendations = sum(1 for item in recommended_item_ids if item in actual_items.values)
    precision = relevant_recommendations / len(recommended_item_ids) if len(recommended_item_ids) > 0 else 0
    
    return precision

# Funzione per calcolare il recall
def calculate_recall(user_id, recommended_items, test_df):
    # Ottieni gli articoli effettivi con cui l'utente ha interagito nel test set (dove is_recommended == 1)
    actual_items = test_df[(test_df['user_id'] == user_id) & (test_df['is_recommended'] == 1)]['app_id']
    
    # Confronta gli articoli raccomandati con quelli effettivi
    recommended_item_ids = [item[0] for item in recommended_items]
    
    # Calcola il recall come frazione di articoli rilevanti che sono stati raccomandati
    relevant_recommendations = sum(1 for item in recommended_item_ids if item in actual_items.values)
    recall = relevant_recommendations / len(actual_items) if len(actual_items) > 0 else 0
    
    return recall

# Funzione per calcolare il F1-Score
def calculate_f1_score(user_id, recommended_items, test_df):
    precision = calculate_precision(user_id, recommended_items, test_df)
    recall = calculate_recall(user_id, recommended_items, test_df)
    
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def calculate_hit_ratio(user_id, recommended_items, test_df, N=10):
    actual_items = test_df[(test_df['user_id'] == user_id) & (test_df['is_recommended'] == 1)]['app_id']
    recommended_item_ids = [item[0] for item in recommended_items][:N]  # Limita alle prime N raccomandazioni
    hit = 1 if any(item in actual_items.values for item in recommended_item_ids) else 0
    return hit

user_id = 22  # esempio di user_id nel test set
recommended_items = get_recommendations(user_id)
weighted_recommendations = get_weighted_recommendations(recommended_items[0], recommended_items[1], N=10)

# Calcola la precisione, recall e F1-Score
precision = calculate_precision(user_id, weighted_recommendations, test_df)
recall = calculate_recall(user_id, weighted_recommendations, test_df)
f1 = calculate_f1_score(user_id, weighted_recommendations, test_df)
hit = calculate_hit_ratio(user_id, weighted_recommendations, test_df)

# Stampa i risultati
print(f"Precisione per l'utente {user_id}: {precision}")
print(f"Recall per l'utente {user_id}: {recall}")
print(f"F1-Score per l'utente {user_id}: {f1}")
print(f"Hit Ratio per l'utente {user_id}: {hit}")


Precisione per l'utente 22: 0.0
Recall per l'utente 22: 0.0
F1-Score per l'utente 22: 0
Hit Ratio per l'utente 22: 0


In [68]:
known_user_metrics = []
for user_id in tqdm(known_users_filtered[0:100]):
    recommended_items = get_weighted_recommendations(*get_recommendations(user_id), N=10)
    precision = calculate_precision(user_id, recommended_items, test_df)
    recall = calculate_recall(user_id, recommended_items, test_df)
    f1 = calculate_f1_score(user_id, recommended_items, test_df)
    hit = calculate_hit_ratio(user_id, recommended_items, test_df)
    known_user_metrics.append((user_id, precision, recall, f1, hit))

 47%|████▋     | 47/100 [00:37<00:42,  1.26it/s]


KeyboardInterrupt: 

In [ ]:
# Calcola la media delle metriche per gli utenti noti
if len(known_user_metrics) > 0:
    avg_precision_known = np.mean([metric[1] for metric in known_user_metrics])
    avg_recall_known = np.mean([metric[2] for metric in known_user_metrics])
    avg_f1_known = np.mean([metric[3] for metric in known_user_metrics])
    avg_hit_known = np.mean([metric[4] for metric in known_user_metrics])
else:
    avg_precision_known, avg_recall_known, avg_f1_known = 0, 0, 0

# Stampa i risultati per gli utenti noti
print("Metriche per gli utenti noti:")
print(f"Precisione media: {avg_precision_known:.4f}")
print(f"Recall medio: {avg_recall_known:.4f}")
print(f"F1-Score medio: {avg_f1_known:.4f}")
print(f"Hit Ratio medio: {avg_hit_known:.4f}")

Metriche per gli utenti noti:
Precisione media: 0.0250
Recall medio: 0.0166
F1-Score medio: 0.0187
Hit Ratio medio: 0.2100


In [79]:
cold_user = cold_start_users[10]
recommended_items = get_weighted_recommendations(*get_recommendations(cold_user, mode='inference'), N=10)
recommended_items

[(631510, 0.2761158407133337),
 (1817070, 0.2368065129442754),
 (941962, 0.23023168637136077),
 (1050460, 0.22192544912770198),
 (705710, 0.22120711977681312),
 (941965, 0.2034982293295735),
 (941952, 0.1923838099822468),
 (941966, 0.19116368590886723),
 (941961, 0.19021823533549728),
 (223790, 0.18888936157904204)]